In [72]:
# Importando Bibliotecas
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import folium
from streamlit_folium import folium_static

In [10]:
df_raw = pd.read_csv('dataset/zomato.csv')

df_raw = pd.read_csv('dataset/zomato.csv')

## Limpando Dados

In [11]:
# Criando Funções:

COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

def Country_Name(country_id):
    return COUNTRIES[country_id]


# -----------------------------------------------


def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
    
# -----------------------------------------------

COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]

# -----------------------------------------------

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df1.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df1.columns = cols_new
    return df

In [12]:
df = df_raw.copy()

# Removendo linhas 'NaN'

df = df.dropna(subset=['Restaurant ID'])
df = df.dropna(subset=['Restaurant Name'])
df = df.dropna(subset=['Cuisines'])
df = df.dropna(subset=['Country Code'])
df = df.dropna(subset=['Address'])
df = df.dropna(subset=['Locality'])
df = df.dropna(subset=['Locality Verbose'])
df = df.dropna(subset=['Longitude'])
df = df.dropna(subset=['Average Cost for two'])
df = df.dropna(subset=['Has Table booking'])
df = df.dropna(subset=['Has Online delivery'])
df = df.dropna(subset=['Price range'])
df = df.dropna(subset=['Aggregate rating'])
df = df.dropna(subset=['Rating color'])
df = df.dropna(subset=['Votes'])
df = df.dropna(subset=['City'])
#df = df.dropna(subset=['City'])

df = df.dropna( )

# Removendo linhas duplicadas
df = df.drop_duplicates()

df = df.drop(columns = ['Switch to order menu'], axis=1)

# Define categorias de preço de acordo com o range
df['Price range'] = df['Price range'].apply(create_price_tye)

# Define o padrão de cores das avaliações
df['Rating color'] = df['Rating color'].apply(color_name)

# --------------------------------------------------------

df['Country Name'] = df['Country Code'].map(Country_Name)
# Definisdo os restaurantes po apenas um tipo de culinaria
df["Cuisines"] = df.loc[:, "Cuisines"].astype(str).apply(lambda x: x.split(",")[0])

# df.info() #valores nulos
#print(df.isnull())
# print(df.isna())

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6929 entries, 0 to 7526
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         6929 non-null   int64  
 1   Restaurant Name       6929 non-null   object 
 2   Country Code          6929 non-null   int64  
 3   City                  6929 non-null   object 
 4   Address               6929 non-null   object 
 5   Locality              6929 non-null   object 
 6   Locality Verbose      6929 non-null   object 
 7   Longitude             6929 non-null   float64
 8   Latitude              6929 non-null   float64
 9   Cuisines              6929 non-null   object 
 10  Average Cost for two  6929 non-null   int64  
 11  Currency              6929 non-null   object 
 12  Has Table booking     6929 non-null   int64  
 13  Has Online delivery   6929 non-null   int64  
 14  Is delivering now     6929 non-null   int64  
 15  Price range          

## Visão Restaurante

In [71]:
df1 = df.copy()
df1 = df1.dropna(subset=['Country Name'])

In [16]:
# df1 = df1.drop('index', axis=1) # Remove index

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [21]:
valor = df1.loc[:, ['Votes', 'Restaurant Name']].groupby('Restaurant Name').count().sort_values(['Votes'],ascending = False).reset_index().iloc[0, 0]
print( f'O nome do restaurante que possui a maior quantidade de avaliações é {valor}' )

O nome do restaurante que possui a maior quantidade de avaliações é Domino's Pizza


### 2. Qual o nome do restaurante com a maior nota média?

In [23]:
valor = df1.loc[:, ['Votes', 'Restaurant Name']].groupby('Restaurant Name').mean().sort_values(['Votes'],ascending = False).reset_index().iloc[0, 0]
print( f'O nome do restaurante com a maior nota média é {valor}' )

O nome do restaurante com a maior nota média é Toit


### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [26]:
valor = df1.loc[:, ['Average Cost for two', 'Restaurant Name']].groupby('Restaurant Name').max().sort_values(['Average Cost for two'],ascending = False).reset_index().iloc[0, 0]
print( f'O nome do restaurante que possui o maior valor de uma prato para duas pessoas é {valor}' )

O nome do restaurante que possui o maior valor de uma prato para duas pessoas é d'Arry's Verandah Restaurant


### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [44]:
valor = df1.loc[df1['Cuisines'] == 'Brazilian', ['Votes', 'Restaurant Name']].groupby('Restaurant Name').mean().sort_values(['Votes'],ascending = True).reset_index().iloc[0, 0]
print( f'O nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação é {valor}' )

O nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação é Cantinho da Gula


###  5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [45]:
valor = df1.loc[(df1['Cuisines'] == 'Brazilian') & (df1['Country Name'] == 'Brazil'), ['Votes', 'Restaurant Name']].groupby('Restaurant Name').mean().sort_values(['Votes'],ascending = False).reset_index().iloc[0, 0]
print( f'O nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação é {valor}' )

O nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação é Veloso


### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [47]:
df1.loc[df1['Has Online delivery'] == 1, ['Votes', 'Restaurant Name']].groupby('Restaurant Name').max().sort_values(['Votes'],ascending = False).reset_index().head(5)

,Restaurant Name,Votes
0,Bawarchi,41333
1,Byg Brewski Brewing Company,17394
2,Hauz Khas Social,13627
3,Paradise,11910
4,Shah Ghouse Hotel & Restaurant,11836


In [48]:
df1.loc[:, ['Votes', 'Restaurant Name']].groupby('Restaurant Name').max().sort_values(['Votes'],ascending = False).reset_index().head(5)

,Restaurant Name,Votes
0,Bawarchi,41333
1,Byg Brewski Brewing Company,17394
2,Toit,15270
3,Truffles,14984
4,Hauz Khas Social,13627


### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas

In [60]:
df1.loc[df1['Has Table booking'] == 1, ['Average Cost for two', 'Restaurant Name']].groupby('Restaurant Name').mean().sort_values(['Average Cost for two'],ascending = False).reset_index().head(5)

,Restaurant Name,Average Cost for two
0,d'Arry's Verandah Restaurant,25000017.0
1,3 Wise Monkeys,450000.0
2,Tapas Club,400000.0
3,Hokkaido Izakaya,350000.0
4,NOBLE by Zab Thai,300000.0


In [61]:
df1.loc[:, ['Average Cost for two', 'Restaurant Name']].groupby('Restaurant Name').mean().sort_values(['Average Cost for two'],ascending = False).reset_index().head(5)

,Restaurant Name,Average Cost for two
0,d'Arry's Verandah Restaurant,25000017.0
1,The Café - Hotel Mulia,1200000.0
2,GIA Restaurant & Bar,700000.0
3,SKYE,600000.0
4,Shabu-Shabu Shaburi,600000.0


### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [68]:
df1.loc[(df1['Cuisines'] == 'Japanese') & (df1['Country Name'] == 'United States of America'), ['Average Cost for two', 'Restaurant Name']].groupby('Restaurant Name').mean().sort_values(['Average Cost for two'],ascending = False).reset_index().head(5)

,Restaurant Name,Average Cost for two
0,Nobu,250.0
1,Morimoto,120.0
2,Roka Akor - Chicago,100.0
3,Uchi,80.0
4,Bamboo Sushi,75.0


In [69]:
df1.loc[(df1['Cuisines'] == 'BBQ') & (df1['Country Name'] == 'United States of America'), ['Average Cost for two', 'Restaurant Name']].groupby('Restaurant Name').mean().sort_values(['Average Cost for two'],ascending = False).reset_index().head(5)

,Restaurant Name,Average Cost for two
0,Lamberts Downtown Barbecue,100.0
1,Russell Street Bar.B.Que,75.0
2,Podnah's Pit BBQ,75.0
3,Redbones Barbecue,75.0
4,Blue Smoke,70.0
